In [33]:
import openai
import pandas as pd

openai.api_key = 'sk-your_api_key'

In [34]:
df = pd.read_csv("Downloads/procedure_info (2).csv")
df_test = df.head(12)
df_test

,id,provider_id,service_id,procedure_id,procedure_description,cash_price,min_allowable,max_allowable,gross_charge,cpt_code
0,0,1,43,969805,mg mammo digital screening bilateral,123.27,130.17,443.77,493.08,77057
1,1,1,43,969805,mg mammo digital screening bilateral,123.27,130.17,443.77,493.08,77067
2,2,1,43,969805,mg mammo digital screening bilateral,123.27,130.17,443.77,493.08,G0202
3,3,1,43,969805,mg mammo digital screening bilateral,123.27,130.17,443.77,493.08,77057
4,4,1,43,969805,mg mammo digital screening bilateral,123.27,130.17,443.77,493.08,77067
5,5,1,43,969805,mg mammo digital screening bilateral,123.27,130.17,443.77,493.08,G0202
6,6,1,5,1167899,ct ankle w/ contrast right,447.15,472.19,1609.73,1788.59,73701
7,7,1,5,1168082,ct hand w/ contrast right,452.98,478.35,1630.74,1811.93,73201
8,8,1,5,1167853,ct angio abdomen,548.84,579.58,1975.83,2195.37,74175
9,9,1,5,1167903,ct ankle w/o contrast left,363.10,383.43,1307.16,1452.40,73700


TEST

In [35]:
from tenacity import (
    retry,
    stop_after_attempt,
    wait_random_exponential,
)  # for exponential backoff


In [36]:
@retry(wait=wait_random_exponential(min=1, max=60), stop=stop_after_attempt(10))
def chatGPT(description):
    name = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "user", "content": f"Create a human readable name for this medical procedure: {description}"}
        ]
    )
    longDescription = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "user", "content": f"What is this MEDICAL PROCEDURE (if you don't know say No Description)? {description}"}
        ]
    )
    return (name.choices[0].message.content, longDescription.choices[0].message.content, description)

In [37]:
noDuplicates = []
duplicateIdxs = []
procedureNameDescription = []
descriptions = []
errorDescriptions = []
idx = 0
length = len(df['procedure_description']) - 1
for description in df['procedure_description']:
    try:
        if description not in noDuplicates:
            procedureNameDescription.append(chatGPT(description))
            noDuplicates.append(description) 
    except: 
        print(f'error for {description}')   
        procedureNameDescription.append((description, 'No Description', description))
    print(f'{(idx/length)*100}% complete -- {idx}')
    idx = idx + 1   

df_final = pd.DataFrame(procedureNameDescription,
                  columns=['procedure_name', 'description', 'old_procedure_name'])
df_final

0.0% complete -- 0
0.014186409419775853% complete -- 1
0.028372818839551707% complete -- 2
0.042559228259327564% complete -- 3
0.056745637679103414% complete -- 4
0.07093204709887928% complete -- 5
0.08511845651865513% complete -- 6
0.09930486593843098% complete -- 7
0.11349127535820683% complete -- 8
0.1276776847779827% complete -- 9
0.14186409419775856% complete -- 10
0.1560505036175344% complete -- 11
0.17023691303731026% complete -- 12
0.18442332245708612% complete -- 13
0.19860973187686196% complete -- 14
0.21279614129663785% complete -- 15
0.22698255071641366% complete -- 16
0.24116896013618955% complete -- 17
0.2553553695559654% complete -- 18
0.2695417789757413% complete -- 19
0.2837281883955171% complete -- 20
0.29791459781529295% complete -- 21
0.3121010072350688% complete -- 22
0.3262874166548447% complete -- 23
0.3404738260746205% complete -- 24
0.3546602354943964% complete -- 25
0.36884664491417224% complete -- 26
0.3830330543339481% complete -- 27
0.3972194637537239% comp

,procedure_name,description,old_procedure_name
0,\n\nBilateral digital screening mammogram for ...,\n\nThis is a medical imaging procedure called...,mg mammo digital screening bilateral
1,\n\nA CT scan with contrast of the right ankle.,\n\nCT ankle with contrast right is a medical ...,ct ankle w/ contrast right
2,"\n\n""Contrast-enhanced CT scan of the right hand""",\n\nCT hand w/ contrast right is a medical ima...,ct hand w/ contrast right
3,\n\nCT Angiography of the Abdomen,\n\nA CT angio of the abdomen is a medical ima...,ct angio abdomen
4,\n\nComputed Tomography of the Left Ankle With...,\n\nCT ankle w/o contrast left is a diagnostic...,ct ankle w/o contrast left
...,...,...,...
1648,"\n\n""X-ray diagnosis of abnormal fistula""",\n\nNo Description.,xray exam of fistula
1649,\n\nOtoacoustic Emissions Screening Automated ...,\n\nOAE screening auto anlys is a medical proc...,oae screening auto anlys
1650,"\n\n""Extraction of Central Venous Access Devic...",\n\nThe MEDICAL PROCEDURE is the removal of a ...,removal cvad w/ port/pump
1651,\n\nComputer-Assisted Digital Screening Mammog...,\n\nNo Description,deleted computerized screen mammo (cad)


In [38]:
df_final.to_csv("namesAndDescriptions1.csv") 

In [44]:
df_for_cleaning = pd.read_csv("namesAndDescriptions1.csv")
df_no_new_lines = df_for_cleaning.replace(["\n", '"'], "", regex=True)
df_no_new_lines

,Unnamed: 0,procedure_name,description,old_procedure_name
0,0,Bilateral digital screening mammogram for brea...,This is a medical imaging procedure called Dig...,mg mammo digital screening bilateral
1,1,A CT scan with contrast of the right ankle.,CT ankle with contrast right is a medical proc...,ct ankle w/ contrast right
2,2,Contrast-enhanced CT scan of the right hand,CT hand w/ contrast right is a medical imaging...,ct hand w/ contrast right
3,3,CT Angiography of the Abdomen,A CT angio of the abdomen is a medical imaging...,ct angio abdomen
4,4,Computed Tomography of the Left Ankle Without ...,CT ankle w/o contrast left is a diagnostic tes...,ct ankle w/o contrast left
...,...,...,...,...
1648,1648,X-ray diagnosis of abnormal fistula,No Description.,xray exam of fistula
1649,1649,Otoacoustic Emissions Screening Automated Anal...,OAE screening auto anlys is a medical procedur...,oae screening auto anlys
1650,1650,Extraction of Central Venous Access Device wit...,The MEDICAL PROCEDURE is the removal of a Cent...,removal cvad w/ port/pump
1651,1651,Computer-Assisted Digital Screening Mammograph...,No Description,deleted computerized screen mammo (cad)


In [45]:
df_no_new_lines.to_csv("namesAndDescriptions1.csv") 

In [46]:
print(len(errorDescriptions))

0
